In [74]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from PIL import Image

from keras.preprocessing.image import load_img 
from keras.preprocessing.image import img_to_array 
from keras.applications.vgg16 import preprocess_input 
# models 
from keras.applications.vgg16 import VGG16 
from keras.models import Model
from keras.applications.vgg16 import preprocess_input 

# clustering and dimension reduction
from sklearn.decomposition import PCA

%load_ext autoreload
%autoreload
from src.dataLoader import dataLoader
from src.utils import image_utils, ismember

from src.preprocessing import imadjust,imagecrop,imagePaddingByShape


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
# dataset = dataLoader("../dataset")

# fileNameList = [str(imgIdx)+".jpg" for imgIdx in range(0,1501)]
# imageFileList = dataset.LoadFileList(fileNameList=fileNameList)
# imageFileList

In [3]:
from os import listdir, rename
from os.path import isfile, join
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import pandas as pd
import random
from sklearn.model_selection import train_test_split
from collections import Counter
from skimage.transform import rotate, AffineTransform, warp
from skimage.util import random_noise

In [108]:
labels = pd.read_csv('labels.csv')
display(labels.head())
labels.shape

,filename,label
0,0.jpg,1
1,1.jpg,1
2,2.jpg,5
3,3.jpg,4
4,4.jpg,1


(1501, 2)

In [117]:
def load_images(labels):
    X = []
    y = []
    for row in labels.itertuples():
        img_path = join('../dataset', row.filename)
        # print(row.filename)
        # print(img_path)
        # img = np.asarray(Image.open(img_path))
        img = load_img(img_path, target_size=(224,224))
        img = np.array(img) 
        # Keep squares for now
        if img.shape[0] == img.shape[1]:
            y.append(row.label)
            X.append(img)
    return X, y


In [128]:
def augment_data(X_train, y_train):
    final_X_train = []
    final_y_train = []
    for i in range(len(X_train)):
        final_X_train.append(X_train[i])
        final_X_train.append(rotate(X_train[i], angle=45, mode = 'wrap'))
        final_X_train.append(np.fliplr(X_train[i]))
        final_X_train.append(np.flipud(X_train[i]))
        final_X_train.append(random_noise(X_train[i],var=0.2**2))
        
        final_y_train += [y_train[i]] * 5
    return final_X_train, final_y_train

In [129]:
def get_accuracy(scores, labels):
    num_data = scores.size(0)
    predicted_labels = scores.argmax(dim=1)
    indicator = (predicted_labels == labels)
    num_matches = indicator.sum()
    return 100*num_matches.float()/num_data  

def get_error( scores , labels ):

    bs=scores.size(0)
    predicted_labels = scores.argmax(dim=1)
    indicator = (predicted_labels == labels)
    num_matches=indicator.sum()
    
    return 1-num_matches.float()/bs   

def eval_on_test_set(test_data, test_label, net, mean, std, bs):

    running_error=0
    running_acc = 0
    num_batches=0

    for i in range(0,500,bs):

        minibatch_data =  test_data[i:i+bs]
        minibatch_label = test_label[i:i+bs]
        
        inputs = (minibatch_data - mean)/std

        scores= net( inputs ) 

        error = get_error( scores.detach() , minibatch_label)
        acc = get_accuracy( scores.detach() , minibatch_label)

        running_error += error.item()
        running_acc += acc.item()

        num_batches+=1

    total_error = running_error/num_batches
    total_acc = running_acc/num_batches
    print(running_error, num_batches)
    print( 'error rate on test set =', total_error*100 ,'percent')
    print('accuracy =', total_acc)

In [130]:
X, y = load_images(labels)
np.array(X).shape

(1501, 224, 224, 3)

In [131]:
Counter(y)

Counter({1: 444, 5: 185, 4: 107, 2: 253, 3: 119, 0: 393})

In [132]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 2, stratify=y)

In [138]:
X_train, y_train = augment_data(X_train, y_train)
X_train.shape

MemoryError: Unable to allocate 1.15 MiB for an array with shape (224, 224, 3) and data type float64

In [134]:
# import torch
# class convnet(torch.nn.Module):

#     def __init__(self, input_channel, hidden_layer, num_class):

#         super(convnet, self).__init__()
      
#         self.conv1a = torch.nn.Conv2d(input_channel, hidden_layer,  kernel_size=5, padding=1 )
#         self.pool1a  = torch.nn.MaxPool2d(2,2)
#         self.conv1b = torch.nn.Conv2d(hidden_layer,  hidden_layer,  kernel_size=5, padding=1 )
#         self.pool1b  = torch.nn.MaxPool2d(2,2)
        
#         self.linear1 = torch.nn.Linear(12100, num_class)

#     def forward(self, x):
#         x = self.conv1a(x)
#         x = torch.nn.functional.relu(x)
#         x = self.pool1a(x)
#         x = self.conv1b(x)
#         x = torch.nn.functional.relu(x)
#         x = self.pool1b(x)
        
#         x = x.view(-1, 12100)
#         x = self.linear1(x)
        
#         return x

In [135]:
import torch
import torch.nn as nn
import torch.optim as optim
from random import randint
import utils
import time

class VGG_convnet(nn.Module):

    def __init__(self, input_channel, num_class):

        super(VGG_convnet, self).__init__()
        
        # block 1:         3 x 50 x 50 --> 64 x 25 x 25        
        self.conv1a = nn.Conv2d(input_channel,   64,  kernel_size=3, padding=1 )
        self.conv1b = nn.Conv2d(64,  64,  kernel_size=3, padding=1 )
        self.pool1  = nn.MaxPool2d(2,2)

        # block 2:         64 x 25 x 25 --> 128 x 8 x 8
        self.conv2a = nn.Conv2d(64,  128, kernel_size=3, padding=1 )
        self.conv2b = nn.Conv2d(128, 128, kernel_size=3, padding=1 )
        self.pool2  = nn.MaxPool2d(2,2)

        # block 3:         128 x 8 x 8 --> 256 x 4 x 4        
        self.conv3a = nn.Conv2d(128, 256, kernel_size=3, padding=1 )
        self.conv3b = nn.Conv2d(256, 256, kernel_size=3, padding=1 )
        self.pool3  = nn.MaxPool2d(2,2)
        
        #block 4:          256 x 4 x 4 --> 512 x 2 x 2
        self.conv4a = nn.Conv2d(256, 512, kernel_size=3, padding=1 )
        self.pool4  = nn.MaxPool2d(2,2)

        # linear layers:   512 x 2 x 2 --> 4608 --> 4096 --> 4096 --> 10
        self.linear1 = nn.Linear(4608, 4608*2)
        self.linear2 = nn.Linear(4608*2,4096)
        self.linear3 = nn.Linear(4096, num_class)


    def forward(self, x):
        # print(x.shape)
        # block 1:         3 x 32 x 32 --> 64 x 16 x 16
        x = self.conv1a(x)
        x = torch.relu(x)
        x = self.conv1b(x)
        x = torch.relu(x)
        x = self.pool1(x)

        # block 2:         64 x 16 x 16 --> 128 x 8 x 8
        x = self.conv2a(x)
        x = torch.relu(x)
        x = self.conv2b(x)
        x = torch.relu(x)
        x = self.pool2(x)

        # block 3:         128 x 8 x 8 --> 256 x 4 x 4
        x = self.conv3a(x)
        x = torch.relu(x)
        x = self.conv3b(x)
        x = torch.relu(x)
        x = self.pool3(x)

        #block 4:          256 x 4 x 4 --> 512 x 2 x 2
        x = self.conv4a(x)
        x = torch.relu(x)
        x = self.pool4(x)

        # linear layers:   512 x 2 x 2 --> 4608 --> 4608*2 --> 4096 --> 10
        # print(x.shape)
        x = x.view(-1, 4608)
        x = self.linear1(x)
        x = torch.relu(x)
        x = self.linear2(x)
        x = torch.relu(x)
        x = self.linear3(x) 
        
        return x

In [136]:
def train_network(X_train, y_train, X_test, y_test):
    X_train = torch.from_numpy(np.array(X_train).astype(np.float32))
    y_train = torch.from_numpy(np.array(y_train).astype(np.float32)).type(torch.LongTensor)
    X_test = torch.from_numpy(np.array(X_test).astype(np.float32))
    y_test = torch.from_numpy(np.array(y_test).astype(np.float32)).type(torch.LongTensor)
    X_train = X_train.view(-1, 3, 50, 50)
    X_test = X_test.view(-1, 3, 50, 50)
    
    my_lr = 0.02
    criterion = torch.nn.CrossEntropyLoss()
    bs = 64
    num_class = 6

    net = VGG_convnet(3, num_class)
    mean = X_train.mean()
    std = X_train.std()
    num_train_data = X_train.size()[0]
    
    for epoch in range(10):
        # create a new optimizer at the beginning of each epoch: give the current learning rate.   
        optimizer=torch.optim.Adam( net.parameters() , lr=my_lr )

        # set the running quatities to zero at the beginning of the epoch
        running_loss=0
        running_error=0
        running_acc = 0
        num_batches=0

        # set the order in which to visit the image from the training set
        shuffled_indices = torch.randperm(num_train_data)

        for count in range(0,num_train_data,bs):

            # Set the gradients to zeros
            optimizer.zero_grad()

            # create a minibatch       
            indices = shuffled_indices[count:count+bs]
            minibatch_data =  X_train[indices]
            minibatch_label =  y_train[indices]


            # normalize the minibatch (this is the only difference compared to before!)
            inputs = (minibatch_data - mean)/std

            # tell Pytorch to start tracking all operations that will be done on "inputs"
            inputs.requires_grad_()

            # forward the minibatch through the net 
            scores=net( inputs ) 

            # Compute the average of the losses of the data points in the minibatch
            loss =  criterion( scores , minibatch_label) 

            # backward pass to compute dL/dU, dL/dV and dL/dW   
            loss.backward()

            # do one step of stochastic gradient descent: U=U-lr(dL/dU), V=V-lr(dL/dU), ...
            optimizer.step()


            # START COMPUTING STATS

            # add the loss of this batch to the running loss
            running_loss += loss.detach().item()

            # compute the error made on this batch and add it to the running error       
            error = get_error( scores.detach() , minibatch_label)
            acc = get_accuracy(scores.detach() , minibatch_label)
            running_error += error.item()
            running_acc += acc.item()

            num_batches+=1        


        # compute stats for the full training set
        total_loss = running_loss/num_batches
        total_error = running_error/num_batches
        total_acc = running_acc/num_batches


        print('epoch=',epoch, '\t loss=', total_loss , '\t error=', total_error*100 ,'percent')
        print('accuracy =', total_acc)
        print(' ')

    eval_on_test_set(X_test, y_test, net, mean, std, bs)

In [137]:
train_network(X_train, y_train, X_test, y_test)

MemoryError: Unable to allocate 6.73 GiB for an array with shape (6000, 224, 224, 3) and data type float64

In [103]:
# choose a picture at random
idx=randint(0, 1500)
label = ('Not Skin', 'Normal', 'Pustule', 'Whitehead', 'Blackhead', 'Cyst',)

print(idx)
im=X[idx]
# X_train, X_test, y_train, y_test
# diplay the picture
image_label = f"label index: {y[idx]}; label is {label[y[idx]]}"
image_utils().plot_image(im,title=image_label, figsize=(5,5))

im = torch.from_numpy(np.array(im).astype(np.float32))
X_train_ = torch.from_numpy(np.array(X_train).astype(np.float32))
X_train_ = X_train_.view(-1, 3, 50, 50)
mean= X_train_.mean()
std= X_train_.std()
# # send to device, rescale, and view as a batch of 1 
# im = im.to(device)
im= (im-mean) / std
im=im.view(1,3,50,50)

# # feed it to the net and display the confidence scores
scores =  net(im) 
probs= torch.softmax(scores, dim=1)
image_utils().show_prob_pimples(probs.cpu())

1427


IndexError: list index out of range

In [106]:
np.array(X).shape

(978, 50, 50, 3)